In [1]:
from google.colab import drive
drive.mount('/content/drive')

import sys
import os
# 프로젝트 폴더명 입력
sys.path.insert(0, os.getcwd()+'/drive/My Drive/Colab Notebooks/recommend_music_by_sentiment_analysis')
import config

Mounted at /content/drive


In [2]:
#@test {"skip": true}
!pip install --quiet --upgrade tensorflow_federated_nightly
!pip install --quiet --upgrade nest_asyncio

import nest_asyncio
nest_asyncio.apply()

     |████████████████████████████████| 501kB 2.6MB/s 
     |████████████████████████████████| 153kB 13.0MB/s 
     |████████████████████████████████| 1.1MB 12.9MB/s 
     |████████████████████████████████| 3.0MB 18.0MB/s 
     |████████████████████████████████| 174kB 23.1MB/s 
     |████████████████████████████████| 390.9MB 36kB/s 
     |████████████████████████████████| 112kB 57.1MB/s 
     |████████████████████████████████| 471kB 51.0MB/s 
     |████████████████████████████████| 10.2MB 44.1MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


In [3]:
import collections
import functools
import os
import time

import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

/usr/local/lib/python3.6/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:43: UserWarning: You are currently using a nightly version of TensorFlow (2.4.0-dev20200925). 
TensorFlow Addons offers no support for the nightly versions of TensorFlow. Some things might work, some other might not. 
If you encounter a bug, do not file an issue on GitHub.
  UserWarning,


In [4]:
DATASET_PATH = config.PATH_CONFIG['DATASET_PATH']

angry = pd.read_excel(DATASET_PATH + "angry.xlsx")
happy = pd.read_excel(DATASET_PATH + "happy.xlsx")
love = pd.read_excel(DATASET_PATH + "love.xlsx")
sad = pd.read_excel(DATASET_PATH + "sad.xlsx")
depressed = pd.read_excel(DATASET_PATH + "depressed.xlsx")

In [5]:
angry.columns=['sentence','label']
happy.columns=['sentence','label']
love.columns=['sentence','label']
sad.columns=['sentence','label']
depressed.columns=['sentence','label']

In [6]:
data=pd.concat([angry,happy,love,sad,depressed])
data = data.reset_index(drop=True)

In [7]:
# 중복 제거
data = data.drop_duplicates()

In [8]:
print(len(data['sentence']), len(data['label']))

16122 16122


In [9]:
# 데이터 전체 저장
data.to_excel(DATASET_PATH + 'All_data.xlsx', sheet_name = 'Sheet1', index = False)

In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(data['sentence'],data['label'],
                                                 test_size=0.33, random_state=42)


In [11]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(10801,)
(5321,)
(10801,)
(5321,)


In [12]:
# lstm 학습에 사용하기 위한 train, test data xlsx 저장
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.33, random_state=42)
train_data.to_excel(DATASET_PATH + 'train_data.xlsx', sheet_name = 'Sheet1', index = False)
test_data.to_excel(DATASET_PATH + 'test_data.xlsx', sheet_name = 'Sheet1', index = False)

In [13]:
data

,sentence,label
0,화나다와 화내다,4
1,여러분물어볼게있는데요.화나다하고속상...,4
2,화나다와 화내다의 구분방법,4
3,‘화나다’는 동사인가요 형용사인가요?,4
4,화나다 / 차이다,4
...,...,...
20786,여자친구랑 헤어질까요?,2
20787,사랑하는데 헤어져야될까요,2
20788,성장 고민..,2
20789,진짜 저만 이렇나요..?,2
